In [1]:
from ultralytics import YOLO

In [3]:
model = YOLO('yolov8s.pt')

results = model.train(data='data.yaml', epochs=100, imgsz=640,device='mps')

100%|██████████| 21.5M/21.5M [00:06<00:00, 3.38MB/s]


New https://pypi.org/project/ultralytics/8.2.83 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.5 🚀 Python-3.12.3 torch-2.3.0 MPS (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_

train: Scanning /Users/muntakim/Developer/Project/barrier/train/labels.cache... 144 images, 0 backgrounds, 0 corrupt: 100%|██████████| 144/144 [00:00<?, ?it/s]
val: Scanning /Users/muntakim/Developer/Project/barrier/valid/labels.cache... 9 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9/9 [00:00<?, ?it/s]

Plotting labels to runs/detect/train4/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train4
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/9 [00:49<?, ?it/s]


KeyboardInterrupt: 

In [2]:
model_test=YOLO('/Users/muntakim/Developer/Project/barrier/runs/detect/train2/weights/best.pt')

In [4]:
out = model_test.export(
    format="engine",
    imgsz=640,
    dynamic=True,
    verbose=False,
    batch=8,
    workspace=2,
    int8=True,
    data="data.yaml"  # COCO, ImageNet, or DOTAv1 for appropriate model task
)

WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics YOLOv8.2.5 🚀 Python-3.12.3 torch-2.3.0 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


In [ ]:
result=model_test.track(source="./test_3.mov",conf=0.3,iou=0.5, show=True)

In [ ]:
from ultralytics import YOLO
from ultralytics.solutions import heatmap
import cv2

model = YOLO('/Users/muntakim/Developer/Project/barrier/runs/detect/train2/weights/best.pt')
cap = cv2.VideoCapture("./test.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
target_fps = 2  # Target frames per second
frame_skip = int(fps / target_fps)
# Video writer
video_writer = cv2.VideoWriter("heatmap_output.avi",
                               cv2.VideoWriter_fourcc(*'mp4v'),
                               target_fps,
                               (w, h))

# Init heatmap
heatmap_obj = heatmap.Heatmap()
heatmap_obj.set_args(colormap=cv2.COLORMAP_PARULA,
                     imw=w,
                     imh=h,
                     view_img=True,
                     heatmap_alpha=0.4,
                     decay_factor=1,
                     shape="circle",
                     classes_names=model.names)
frame_count = 0
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    frame_count += 1
    if frame_count % frame_skip != 0:
        continue
    tracks = model.track(im0,conf=0.5,iou=0.5, persist=True, show=False)
    annotated_frame = tracks[0].plot()
    im0 = heatmap_obj.generate_heatmap(annotated_frame, tracks)
    video_writer.write(im0)

cap.release()
video_writer.release()

cv2.destroyAllWindows()

In [ ]:
from ultralytics import YOLO
from ultralytics.solutions import heatmap
import cv2
from shapely.geometry import Polygon
from collections import defaultdict
import numpy as np

# Initialize YOLO model
model = YOLO('/Users/muntakim/Developer/Project/barrier/runs/detect/train2/weights/best.pt')
model_2=YOLO('yolov8n-seg.pt')

def detect_red(frame,name):
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define range of red color in HSV
    lower_red = np.array([170, 180, 100]) # Red Range
    upper_red = np.array([180, 255, 255]) # Red Range

    # Threshold the HSV image to get only red colors
    mask = cv2.inRange(hsv, lower_red, upper_red)
    
    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame,frame, mask= mask)
    cv2.imshow(name+"_mask",mask)
    # print(cv2.countNonZero(mask))
    # Check if any red pixels are present
    if cv2.countNonZero(mask) > 600:
        return True
    else:
        return False
        
# Load video
cap = cv2.VideoCapture("./test_3.mov")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Define area of interest (AOI) coordinates (for example, a rectangle)
m_aoi_x1, m_aoi_y1, m_aoi_x2, m_aoi_y2 = 800, 1550, 2700, 1690  # Define AOI coordinates (top-left and bottom-right)


# aoi_x1, aoi_y1, aoi_x2, aoi_y2 = 800, 1400, 2700, 1550



# Target frames per second
target_fps = 2
frame_skip = int(fps / target_fps)

# # Video writer
# video_writer = cv2.VideoWriter("heatmap_output.avi",
#                                cv2.VideoWriter_fourcc(*'mp4v'),
#                                target_fps,
#                                (w, h))

# Init heatmap
# heatmap_obj = heatmap.Heatmap()
# heatmap_obj.set_args(colormap=cv2.COLORMAP_PARULA,
#                      imw=w,
#                      imh=h,
#                      view_img=False,
#                      heatmap_alpha=0.2,
#                      decay_factor=0.5,
#                      shape="circle",
#                      classes_names=model.names)

frame_count = 0
aoi_polygon = Polygon([(m_aoi_x1, m_aoi_y1), (m_aoi_x2, m_aoi_y1), (m_aoi_x2, m_aoi_y2), (m_aoi_x1, m_aoi_y2)])
left_count=0
right_count=0
barrier_close_count=0
change_count = 0
frame_stack=[]
frame_stack_left=[]
frame_stack_right=[]

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        frame_count=0
        print("Video frame is empty or video processing has been successfully completed.")
        cap.set(1,0)
    frame_count += 1
    if frame_count % frame_skip != 0:
        continue
        
    tracks = model.track(im0, conf=0.5, iou=0.5, persist=True,retina_masks=True,device='mps', show=False)
    results = model_2.track(tracks[0].plot(),conf=0.25,iou=.5,classes=[0,1,2,3,5,7,15,16,17,18,19,20,21,22,23],retina_masks=True,device='mps',persist=True)
    annotated_frame=results[0].plot()
    cv2.rectangle(annotated_frame, (m_aoi_x1, m_aoi_y1), (m_aoi_x2, m_aoi_y2), (0, 0, 255), 5)
    
    x_l, y_l, w_l, h_l = 720, 1280, 100, 90
    x1_l, y1_l = int(x_l-w_l/2), int(y_l-h_l/2)
    x2_l, y2_l = int(x_l+w_l/2), int(y_l+h_l/2)
    cv2.rectangle(im0, (x1_l, y1_l), (x2_l, y2_l), (255, 0, 0), 5)
    cropped_left = im0[y1_l:y2_l, x1_l:x2_l]
    red_detected_left = detect_red(cropped_left,'left')
    frame_stack_left.append(red_detected_left)
    
    if len(frame_stack_left) >= 2:
    # Check if the last two elements of the stack are different
        if frame_stack_left[-1] != frame_stack_left[-2]:
            left_count += 1
    # print(red_detected_left)
    
    cv2.imshow("left",cropped_left)
    x_r, y_r, w_r, h_r = 900, 1280, 100, 90
    x1_r, y1_r = int(x_r-w_r/2), int(y_r-h_r/2)
    x2_r, y2_r = int(x_r+w_r/2), int(y_r+h_r/2)
    cv2.rectangle(im0, (x1_r, y1_r), (x2_r, y2_r), (255, 0, 0), 5)
    cropped_right = im0[y1_r:y2_r, x1_r:x2_r]
    red_detected_right = detect_red(cropped_right,'right')
    frame_stack_right.append(red_detected_right)
    if len(frame_stack_right) >= 2:
    # Check if the last two elements of the stack are different
        if frame_stack_right[-1] != frame_stack_right[-2]:
            right_count += 1
            
    # print(red_detected_right)
    cv2.imshow("right",cropped_right)
    # Detect if the detection is in the AOI
    for track in tracks:
        for obj in track.boxes.xyxy:
            bbox_polygon = Polygon([(obj[0], obj[1]), (obj[2], obj[1]), (obj[2], obj[3]), (obj[0], obj[3])])
            if bbox_polygon.intersects(aoi_polygon):
                # Detection is within AOI
                cv2.rectangle(annotated_frame, (m_aoi_x1, m_aoi_y1), (m_aoi_x2, m_aoi_y2), (0, 0, 255), 5)
                text = "BOOM GATE IS CLOSED"
                frame_state=True
                frame_stack.append(frame_state)
                if len(frame_stack) >= 2:
                    # Check if the last two elements of the stack are different
                    if frame_stack[-1] != frame_stack[-2]:
                        change_count += 1
                text_position = (m_aoi_x1+500, m_aoi_y1-50)  # Adjust the offset as needed
                # Add text above the rectangle
                cv2.putText(annotated_frame, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 4, (255, 0, 0), 5)
            else:
                # Detection is outside AOI
                cv2.rectangle(annotated_frame, (m_aoi_x1, m_aoi_y1), (m_aoi_x2, m_aoi_y2), (0, 255, 0), 5)  
                text = "BOOM GATE IS OPEN"
                text_position = (m_aoi_x1+500, m_aoi_y1-50)  # Adjust the offset as needed
                # Add text above the rectangle
                cv2.putText(annotated_frame, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 0, 0), 5)
                frame_state=False
                frame_stack.append(frame_state)
    # Generate heatmap with AOI
    # im0 = heatmap_obj.generate_heatmap(annotated_frame, tracks)
    text=f"Annoted_left_{left_count}_right_{right_count}_barrier_close_count_{change_count}"
    cv2.putText(annotated_frame, text, (500, 90), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 0, 0), 5)
    cv2.imshow("annotated",annotated_frame)
    if cv2.waitKey(1) and 0xFF == ord('q'):  
        break
    # Write frame to video
    # video_writer.write(im0)
    
del frame_stack

cap.release()
video_writer.release()
cv2.destroyAllWindows()



0: 384x640 1 BOOMGATE, 15.8ms
Speed: 2.4ms preprocess, 15.8ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.9ms
Speed: 2.5ms preprocess, 22.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BOOMGATE, 9.4ms
Speed: 2.3ms preprocess, 9.4ms inference, 4.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.1ms
Speed: 2.3ms preprocess, 10.1ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BOOMGATE, 13.3ms
Speed: 2.8ms preprocess, 13.3ms inference, 9.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.2ms
Speed: 2.1ms preprocess, 13.2ms inference, 4.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BOOMGATE, 16.1ms
Speed: 2.0ms preprocess, 16.1ms inference, 5.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 16.6ms
Speed: 2.2ms preprocess, 16.6ms inference, 2.4ms postpr

2024-05-13 21:51:34.693 python[91265:819017] Warning: Window move completed without beginning


0: 384x640 1 BOOMGATE, 12.7ms
Speed: 3.0ms preprocess, 12.7ms inference, 7.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 13.6ms
Speed: 2.1ms preprocess, 13.6ms inference, 8.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BOOMGATE, 12.4ms
Speed: 2.8ms preprocess, 12.4ms inference, 7.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 13.8ms
Speed: 2.3ms preprocess, 13.8ms inference, 10.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BOOMGATE, 12.3ms
Speed: 3.0ms preprocess, 12.3ms inference, 6.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 13.9ms
Speed: 2.7ms preprocess, 13.9ms inference, 11.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BOOMGATE, 12.4ms
Speed: 3.0ms preprocess, 12.4ms inference, 6.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 13.8ms
Speed: 2.2ms preprocess, 13.8ms inference, 17.0ms postprocess per image at shape (1, 3, 384,

2024-05-13 21:51:38.086 python[91265:819017] Warning: Window move completed without beginning


0: 384x640 1 BOOMGATE, 12.1ms
Speed: 3.1ms preprocess, 12.1ms inference, 6.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 13.6ms
Speed: 2.2ms preprocess, 13.6ms inference, 22.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BOOMGATE, 12.3ms
Speed: 2.9ms preprocess, 12.3ms inference, 5.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 9.8ms
Speed: 2.2ms preprocess, 9.8ms inference, 15.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BOOMGATE, 12.4ms
Speed: 2.9ms preprocess, 12.4ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 13.3ms
Speed: 2.1ms preprocess, 13.3ms inference, 10.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BOOMGATE, 12.4ms
Speed: 3.0ms preprocess, 12.4ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 10.5ms
Speed: 2.4ms preprocess, 10.5ms inference, 22.6ms postprocess per image at shape (1, 3, 384, 

2024-05-13 21:51:41.457 python[91265:819017] Warning: Window move completed without beginning



0: 384x640 1 BOOMGATE, 12.3ms
Speed: 3.0ms preprocess, 12.3ms inference, 14.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 10.3ms
Speed: 2.0ms preprocess, 10.3ms inference, 9.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BOOMGATE, 12.5ms
Speed: 2.3ms preprocess, 12.5ms inference, 7.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 13.5ms
Speed: 2.4ms preprocess, 13.5ms inference, 11.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BOOMGATE, 12.2ms
Speed: 2.9ms preprocess, 12.2ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 10.7ms
Speed: 2.2ms preprocess, 10.7ms inference, 23.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 BOOMGATE, 12.4ms
Speed: 2.8ms preprocess, 12.4ms inference, 6.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 13.9ms
Speed: 2.3ms preprocess, 13.9ms inference, 24.7ms postprocess per image at shape (1, 3, 38

In [1]:
from ultralytics import YOLO
from ultralytics.solutions import heatmap
import cv2
from shapely.geometry import Polygon
from collections import defaultdict
import numpy as np
# from tb_device_mqtt import TBDeviceMqttClient
import base64
# from jtop import jtop
# import json

## Initialize the Innova-iot 
# client = TBDeviceMqttClient("10.100.0.183", username="XNvsJiIiQGbguIZ3w6Ly")
# client.connect()

# Initialize YOLO model
model = YOLO('./best.pt') # BoomGate 
model_2=YOLO('yolov8n-seg.pt') #Humans, cars & etc.


# Load video
cap = cv2.VideoCapture("./test_3.mov")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Define area of interest (AOI) coordinates (for example, a rectangle)
# m_aoi_x1, m_aoi_y1, m_aoi_x2, m_aoi_y2 = 800, 1550, 2700, 1690  # Define AOI coordinates (top-left and bottom-right)

m_aoi_x1, m_aoi_y1, m_aoi_x2, m_aoi_y2 = 300, 500, 800, 570 

# aoi_x1, aoi_y1, aoi_x2, aoi_y2 = 800, 1400, 2700, 1550



# Target frames per second
target_fps = 1
frame_skip = int(fps / target_fps)


# Init heatmap
# heatmap_obj = heatmap.Heatmap()
# heatmap_obj.set_args(colormap=cv2.COLORMAP_PARULA,
#                      imw=w,
#                      imh=h,
#                      view_img=False,
#                      heatmap_alpha=0.2,
#                      decay_factor=0.5,
#                      shape="circle",
#                      classes_names=model.names)

def detect_red(frame):
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define range of red color in HSV
    lower_red = np.array([170, 180, 100]) # Red Range
    upper_red = np.array([180, 255, 255]) # Red Range

    # Threshold the HSV image to get only red colors
    mask = cv2.inRange(hsv, lower_red, upper_red)
    
    # Bitwise-AND mask and original image
    res = cv2.bitwise_and(frame,frame, mask= mask)
    # cv2.imshow(name+"_mask",mask)
    # print()
    # print(cv2.countNonZero(mask))
    # Check if any red pixels are present
    if cv2.countNonZero(mask) > 50:
        return True
    else:
        return False


frame_count = 0
aoi_polygon = Polygon([(m_aoi_x1, m_aoi_y1), (m_aoi_x2, m_aoi_y1), (m_aoi_x2, m_aoi_y2), (m_aoi_x1, m_aoi_y2)])

left_count=0
right_count=0
change_count = 0
frame_stack=[]
frame_stack_left=[]
frame_stack_right=[]


while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    frame_count += 1
    if frame_count % frame_skip != 0:
        continue
    im0=cv2.resize(im0,(1200,720))
    tracks = model.track(im0, conf=0.5, iou=0.5, persist=True,device='cuda',retina_masks=True, show=False)
    results = model_2.track(tracks[0].plot(),conf=0.25,iou=.5,classes=[0,1,2,3,5,7,15,16,17,18,19,20,21,22,23],device='cuda',retina_masks=True,persist=True)
    annotated_frame=results[0].plot()
    cv2.rectangle(annotated_frame, (m_aoi_x1, m_aoi_y1), (m_aoi_x2, m_aoi_y2), (0, 0, 255), 5) # BOOM GATE AOI

    x_l, y_l, w_l, h_l = 225, 430, 25, 25 # AOI for Light left
    x1_l, y1_l = int(x_l-w_l/2), int(y_l-h_l/2)
    x2_l, y2_l = int(x_l+w_l/2), int(y_l+h_l/2)
    cv2.rectangle(im0, (x1_l, y1_l), (x2_l, y2_l), (255, 0, 0), 1)
    cropped_left = im0[y1_l:y2_l, x1_l:x2_l]
    red_detected_left = detect_red(cropped_left)
    frame_stack_left.append(red_detected_left)
    
    if len(frame_stack_left) >= 2:
    # Check if the last two elements of the stack are different
        if frame_stack_left[-1] != frame_stack_left[-2]:
            left_count += 1
    # print(left_count)
    
    x_r, y_r, w_r, h_r = 285, 430, 25, 25 # AOI for Light right
    x1_r, y1_r = int(x_r-w_r/2), int(y_r-h_r/2)
    x2_r, y2_r = int(x_r+w_r/2), int(y_r+h_r/2)
    cv2.rectangle(im0, (x1_r, y1_r), (x2_r, y2_r), (255, 0, 0), 1)
    cropped_right = im0[y1_r:y2_r, x1_r:x2_r]
    red_detected_right = detect_red(cropped_right)
    frame_stack_right.append(red_detected_right)
    
    if len(frame_stack_right) >= 2:
    # Check if the last two elements of the stack are different
        if frame_stack_right[-1] != frame_stack_right[-2]:
            right_count += 1
    # client.send_telemetry()
    # print(right_count)
    # Detect if the detection is in the AOI
    for track in tracks:
        for obj in track.boxes.xyxy:
            bbox_polygon = Polygon([(obj[0], obj[1]), (obj[2], obj[1]), (obj[2], obj[3]), (obj[0], obj[3])])
            if bbox_polygon.intersects(aoi_polygon):
                # Detection is within AOI
                cv2.rectangle(annotated_frame, (m_aoi_x1, m_aoi_y1), (m_aoi_x2, m_aoi_y2), (0, 0, 255), 5)
                text = "BOOM GATE IS CLOSED"
                text_position = (m_aoi_x1+100, m_aoi_y1-10)  # Adjust the offset as needed

                # Add text above the rectangle
                cv2.putText(annotated_frame, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

                ret, buffer = cv2.imencode('.jpg', annotated_frame)

                # Convert the image buffer to base64
                base64_image = base64.b64encode(buffer)
                # Convert base64 bytes to a string
                base64_string =base64_image.decode('utf-8')
                frame_state=True
                frame_stack.append(frame_state)
                if len(frame_stack) >= 2:
                    # Check if the last two elements of the stack are different
                    if frame_stack[-1] != frame_stack[-2]:
                        change_count += 1
                with jtop() as jetson:
                    stats=dict(jetson.stats)
                    stats['time']=str(jetson.stats['time'])
                    stats['uptime']=str(jetson.stats['uptime'])
                    stats['BOOMGATE_STATUS']=False
                    stats['frame']=base64_string
                    stats['led_left_on_count']=left_count
                    stats['led_right_on_count']=right_count
                    client.send_telemetry(
                        stats
                    ) 
            else:
                # Detection is outside AOI
                cv2.rectangle(annotated_frame, (m_aoi_x1, m_aoi_y1), (m_aoi_x2, m_aoi_y2), (0, 255, 0), 5)  
                text = "BOOM GATE IS OPEN"
                text_position = (m_aoi_x1+100, m_aoi_y1-10)  # Adjust the offset as needed
                # Add text above the rectangle
                cv2.putText(annotated_frame, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                ret, buffer = cv2.imencode('.jpg', annotated_frame)
                # Convert the image buffer to base64
                base64_image = base64.b64encode(buffer)
                # Convert base64 bytes to a string
                base64_string = base64_image.decode('utf-8')
                with jtop() as jetson:
                    stats=dict(jetson.stats)
                    stats['time']=str(jetson.stats['time'])
                    stats['uptime']=str(jetson.stats['uptime'])
                    stats['BOOMGATE_STATUS']=True
                    stats['frame']=base64_string
                    stats['led_left_on_count']=left_count
                    stats['led_rled_right_on_countight']=right_count
                    client.send_telemetry(
                        stats
                    )
                frame_state=False
                frame_stack.append(frame_state)
#     # Generate heatmap with AOI
#     # im0 = heatmap_obj.generate_heatmap(annotated_frame, tracks)
    # text=f"Annoted_left_{left_count}_right_{right_count}_barrier_close_count_{change_count}"
    # cv2.putText(annotated_frame, text, (200, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    # cv2.imshow("Annoted",annotated_frame)
    # if cv2.waitKey(1) and 0xFF == ord('q'):  
    #     break
    # Write frame to video
#     video_writer.write(im0)

# cap.release()
# video_writer.release()
# cv2.destroyAllWindows()
while True:
    pass



0: 384x640 1 BOOMGATE, 214.7ms
Speed: 8.7ms preprocess, 214.7ms inference, 324.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 139.8ms
Speed: 2.4ms preprocess, 139.8ms inference, 5.5ms postprocess per image at shape (1, 3, 384, 640)
0
0

0: 384x640 1 BOOMGATE, 12.2ms
Speed: 2.8ms preprocess, 12.2ms inference, 6.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.1ms
Speed: 1.8ms preprocess, 13.1ms inference, 5.6ms postprocess per image at shape (1, 3, 384, 640)
0
0

0: 384x640 1 BOOMGATE, 12.6ms
Speed: 2.8ms preprocess, 12.6ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.5ms
Speed: 1.8ms preprocess, 10.5ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)
0
0

0: 384x640 1 BOOMGATE, 13.6ms
Speed: 1.8ms preprocess, 13.6ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.3ms
Speed: 1.8ms preprocess, 10.3ms inf